In [1]:
import ROOT
from ROOT import TChain
from ROOT import TCanvas
from ROOT import TLegend
from ROOT import RDataFrame
from ROOT import RVec
from ROOT import kRed, kBlue, gPad
import numpy as np
#import sys
#sys.path.append("/home/NEUTRINO/croselli/.local/lib/python3.9/site-packages")

import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

ROOT.gSystem.Load("libROOTDataFrame")
#%jsroot
#load the tree
#df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/croselli/root_macros/Rough_tree_31_10.root")
#df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/croselli/root_macros/Rough_tree_04_11_IN.root")
#df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/croselli/root_macros/Rough_tree_PROVA_IN.root")
#df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/croselli/root_macros/Rough_tree_05_11_IN.root")
#df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/croselli/root_macros/Rough_tree_08_11.root")

#df = RDataFrame("Rough_tree", "Rough_tree_10_11_GS.root")
#df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/croselli/root_macros/Rough_tree_16_11_GS.root")
df = RDataFrame("Rough_tree", "/storage/gpfs_data/neutrino/users/croselli/root_macros/Solid_Hydrogen2/Rough_tree_18_11.root")

In [2]:
#select the TRUE events with a antimuon and a nuetron in the final state 
ROOT.gInterpreter.Declare("""
bool IsMuonNeutron(const ROOT::VecOps::RVec<int>& pdgs) {
    if (pdgs.size() != 2) return false;
    return ((pdgs[0] == -13 && pdgs[1] == 2112) || (pdgs[0] == 2112 && pdgs[1] == -13));
}
""")

#select the events with a successfull reconstruction
ROOT.gInterpreter.Declare("""
    bool Reco_ok( const Int_t& N_tracks, ROOT::VecOps::RVec<double> mu_pxreco, ROOT::VecOps::RVec<double> mu_pyreco, ROOT::VecOps::RVec<double> mu_pzreco, double mu_Preco ) {
                          if (N_tracks == 0) return false; 
                          if (mu_pxreco.empty()) return false;
                          if (mu_pzreco[0] < 0.0) return false;
                          //if (mu_Preco == 1.000000 || mu_Preco == 0.000000) return false;
                          if (std::abs(mu_Preco - 1.0) < 1e-9 || std::abs(mu_Preco - 0.0) < 1e-9) return false;
                          if (std::abs(mu_Preco - 0.0) < 1e-9 || std::abs(mu_Preco - 0.0) < 1e-9) return false;
                          return true; 
                          }
""")


#Filter on neutron predicetd position
ROOT.gInterpreter.Declare("""
using namespace ROOT::VecOps;
                          
RVec<bool> neutron_correspondence(RVec<double> hit_pred, RVec<double> x_ecal, RVec<double> y_ecal, RVec<double> z_ecal ){

double r = 100.0; //(mm) radius of the cilinder almost half of the ECAL trapezoidal module
//double r = 200.0;
//double r = 180.0;
//double r = 150.0;
// double r = 140.0;                        
//double r = 120.0;
                          

//cilinder axis
double x1 = hit_pred[0], y1 = hit_pred[1], z1 = hit_pred[2];
double x2 = hit_pred[3], y2 = hit_pred[4], z2 = hit_pred[5];

double vx = x2 - x1;
double vy = y2 - y1;
double vz = z2 - z1;
double vlen2 = vx*vx + vy*vy + vz*vz; 

//return vector
size_t n = x_ecal.size();
RVec<bool> inside(n, false);
                          
for (size_t i = 0; i < n; ++i) {
    
    //cluster point
    double qx = x_ecal[i] - x1;
    double qy = y_ecal[i] - y1;
    double qz = z_ecal[i] - z1;
                          
    // Parametro t della proiezione
    double t = (qx*vx + qy*vy + qz*vz) / vlen2;

    if (t < 0.0 || t > 1.0) {
        inside[i] = false;
        continue;
    }
                          
    // Punto proiettato sull'asse
    double px = x1 + t * vx;
    double py = y1 + t * vy;
    double pz = z1 + t * vz;
                          
    // Distanza punto-asse
    double dx = x_ecal[i] - px;
    double dy = y_ecal[i] - py;
    double dz = z_ecal[i] - pz;
    double dist2 = dx*dx + dy*dy + dz*dz;
                          
    inside[i] = (dist2 <= r*r);
}

return inside; 
}                          
""")



True

In [3]:
ROOT.gInterpreter.Declare("""
ROOT::VecOps::RVec<double> Predicted_hit(double px, double py, double pz,
                                         double x0, double y0, double z0)
{

    // ECAL cylinder dimensions (mm)
    const double r_out = 2230.0;
    const double r_in  = 2000.0;
    const double x_min = -2150.0;
    const double x_max =  2150.0;

    // Cylinder center
    const double cx = 0.0;
    const double cy = -2384.73;
    const double cz = 23910.0;

    // Normalize direction
    double norm = std::sqrt(px*px + py*py + pz*pz);
    if (norm == 0) return {0,0,0,0,0,0};

    double nx = px / norm;
    double ny = py / norm;
    double nz = pz / norm;

                          
    //cilinder intersection
    auto intersect = [&](double R)
    {
        // Shift ray origin into cylinder frame
        double Y = y0 - cy;
        double Z = z0 - cz;

        // Quadratic coefficients for (y^2 + z^2 = R^2)
        double A = ny*ny + nz*nz;
        double B = 2*(Y*ny + Z*nz);
        double C = Y*Y + Z*Z - R*R;

        double D = B*B - 4*A*C;
        if (D < 0 || A == 0)
            return std::vector<double>{};

        double sqrtD = std::sqrt(D);
        double t1 = (-B + sqrtD)/(2*A);
        double t2 = (-B - sqrtD)/(2*A);

        double t = 1e99;
        if (t1 > 0) t = std::min(t, t1);
        if (t2 > 0) t = std::min(t, t2);

        if (t == 1e99)
            return std::vector<double>{};

        // Intersection point
        double xi = x0 + t*nx;
        double yi = y0 + t*ny;
        double zi = z0 + t*nz;
        return std::vector<double>{xi, yi, zi};
    };
                          

    // plane intersection
    auto intersect_plane = [&](double x_plane) {
                          
        if (nx == 0) return std::vector<double>{0.0,0.0,0.0}; 
        double t = (x_plane - x0) / nx;
        if (t <= 0) return std::vector<double>{0.0,0.0,0.0};  // plane behind start

        double xi = x_plane;
        double yi = y0 + t*ny;
        double zi = z0 + t*nz;
        return std::vector<double>{xi, yi, zi};
    };

                      
    // Compute inner & outer intersections with cylinder
    auto in  = intersect(r_in);
    auto out = intersect(r_out);

    // If intersection missing
    if (in.empty())  in  = {0,0,0};
    if (out.empty()) out = {0,0,0};

    ROOT::VecOps::RVec<double> result;
    result.insert(result.end(), in.begin(), in.end());
    result.insert(result.end(), out.begin(), out.end());
    
    //Endcpas
        if (result[0] < x_min) {
        double xA = -1690;
        double xB = -1690 - 460.0;

        auto pA = intersect_plane(xA);
        auto pB = intersect_plane(xB);

        ROOT::VecOps::RVec<double> res_planes;
        res_planes.insert(res_planes.end(), pA.begin(), pA.end());
        res_planes.insert(res_planes.end(), pB.begin(), pB.end());
        return res_planes;  
    }

        if (result[0] > x_max) {
        double xA = 1690;
        double xB = 1690 + 460.0;

        auto pA = intersect_plane(xA);
        auto pB = intersect_plane(xB);

        ROOT::VecOps::RVec<double> res_planes;
        res_planes.insert(res_planes.end(), pA.begin(), pA.end());
        res_planes.insert(res_planes.end(), pB.begin(), pB.end());
        return res_planes;  
    }                                 
                                           
    return result;
                          
}
""")

True

In [4]:
df_extended = (df.Define("hit_pred2", "Predicted_hit( n_px_pred, n_py_pred, n_pz_pred, vertex_x, vertex_y, vertex_z)")
                 .Define("inside", "neutron_correspondence( hit_pred, x_ecal, y_ecal, z_ecal)")
                 .Define("n_inside", "Sum(inside)")
)

In [5]:
true_filter = 'IsMuonNeutron(primaries_PDG) && target == "tgt:1000010010" && st_proc_type == "proc:Weak[CC],QES"'
volume_filter = "vertex_z > 22850"
reco_filter = "Reco_ok(N_tracks, mu_pxreco, mu_pyreco, mu_pzreco, mu_Preco )"
track_filter = "N_tracks == 1"
space_filter = "n_inside > 0"

# nu_E_filter = "nuE > 5"
# n_space_filter = "n_inside > 0"
# n_time_filter = "t_match > 0 "

df_flag = (df_extended.Define("true_flag", f"({true_filter}) ? 1 : 0")
                      .Define("volume_flag", f"({volume_filter}) ? 1 : 0")
                      .Define("reco_flag", f"({reco_filter}) ? 1 : 0")
                      #.Define("reco_ok", "Reco_ok(N_tracks, mu_pxreco, mu_pyreco, mu_pzreco, mu_Preco )")
                      .Define("track_flag", f"({track_filter}) ? 1 : 0")
                      .Define("space_flag", f"({space_filter}) ? 1 : 0")
)

In [6]:
df_check = df_flag.Filter("true_flag == 1 && reco_flag == 1 && space_flag == 1").Display(["hit_pred2"],100).Print()

+-------+--------------+
| Row   | hit_pred2    | 
+-------+--------------+
| 37    | -30.350323   | 
|       | -2560.630963 | 
|       | 25902.249696 | 
|       | -461.600320  | 
|       | -2705.680489 | 
|       | 26116.782904 | 
+-------+--------------+
| 227   | 648.067739   | 
|       | -3741.884497 | 
|       | 25379.058090 | 
|       | 608.559520   | 
|       | -3930.464188 | 
|       | 25517.359891 | 
+-------+--------------+
| 292   | 1024.980440  | 
|       | -386.324918  | 
|       | 23830.143087 | 
|       | 1118.957013  | 
|       | -154.738408  | 
|       | 23903.876308 | 
+-------+--------------+
| 523   | 751.286662   | 
|       | -2141.710377 | 
|       | 25895.180461 | 
|       | 712.338497   | 
|       | -1902.252231 | 
|       | 26087.180563 | 
+-------+--------------+
| 773   | -1690.000000 | 
|       | -2126.163913 | 
|       | 24709.473905 | 
|       | -2150.000000 | 
|       | -1185.280870 | 
|       | 25406.210382 | 
+-------+--------------+
| 1064  | 1015.4658

In [11]:
df_check = df_flag.Filter("true_flag == 0 && reco_flag == 1 && space_flag == 0").Display(["hit_pred2"],100).Print()

+-----+--------------+
| Row | hit_pred2    | 
+-----+--------------+
| 1   | -1690.000000 | 
|     | -1189.110877 | 
|     | 23199.699246 | 
|     | -2150.000000 | 
|     | -944.058381  | 
|     | 23528.771928 | 
+-----+--------------+
| 2   | 966.928564   | 
|     | -1057.040143 | 
|     | 25405.740500 | 
|     | 1221.184188  | 
|     | -881.997108  | 
|     | 25557.632803 | 
+-----+--------------+
| 4   | 374.143290   | 
|     | -708.260203  | 
|     | 22819.381359 | 
|     | 393.607585   | 
|     | -383.308745  | 
|     | 22926.530144 | 
+-----+--------------+
| 5   | -1237.752592 | 
|     | -1991.805065 | 
|     | 25871.022691 | 
|     | -1374.365986 | 
|     | -1688.972657 | 
|     | 26028.683959 | 
+-----+--------------+
| 6   | 663.458356   | 
|     | -4309.364748 | 
|     | 24453.857596 | 
|     | 679.070062   | 
|     | -4536.148775 | 
|     | 24496.768485 | 
+-----+--------------+
| 7   | 824.710378   | 
|     | -3192.036721 | 
|     | 25739.823997 | 
|     | 1195.293304  | 

In [7]:
%jsroot

In [8]:
# --- Parametri cilindri (in mm) ---
r_out, r_in = 2230.0, 2000.0
x_min, x_max = -2150.0, 2150.0

# Centro del cilindro
x_c, y_c, z_c = 0.0, -2384.73, 23910.0

#event_index = 10


event_index = 227 #good
#event_index = 1

evt_df = df_flag.Range(event_index,event_index+1)
# evt_df = (evt_df.Define("x1", "hit_pred[0]")
#           .Define("y1", "hit_pred[1]")
#           .Define("z1", "hit_pred[2]")
#           .Define("x2", "hit_pred[3]")
#           .Define("y2", "hit_pred[4]")
#           .Define("z2", "hit_pred[5]"))

evt_df = (evt_df.Define("x1", "hit_pred2[0]")
          .Define("y1", "hit_pred2[1]")
          .Define("z1", "hit_pred2[2]")
          .Define("x2", "hit_pred2[3]")
          .Define("y2", "hit_pred2[4]")
          .Define("z2", "hit_pred2[5]")
          .Define("px_n", "n_px[0]")
          .Define("py_n", "n_py[0]")
          .Define("pz_n", "n_pz[0]")
          .Define("true_mom", "sqrt(px_n*px_n + py_n*py_n + pz_n*pz_n)" )
          .Define("px_end", "px_n/true_mom")
          .Define("py_end", "py_n/true_mom")
          .Define("pz_end", "pz_n/true_mom")
          .Define("pred_mom", "sqrt(n_px_pred*n_px_pred + n_py_pred*n_py_pred + n_pz_pred*n_pz_pred)" )
          .Define("px_end2", "n_px_pred/pred_mom")
          .Define("py_end2", "n_py_pred/pred_mom")
          .Define("pz_end2", "n_pz_pred/pred_mom")
          
          )

vtx_pos = evt_df.AsNumpy(["vertex_x","vertex_y","vertex_z"])
p1 = evt_df.AsNumpy(["x1","y1","z1"])
p2 = evt_df.AsNumpy(["x2","y2","z2"])
traj_point = evt_df.AsNumpy(["true_x_pos", "true_y_pos", "true_z_pos"])
ecal_point = evt_df.AsNumpy(["x_ecal", "y_ecal", "z_ecal"])
seg_point = evt_df.AsNumpy(["true_n_startX", "true_n_startY", "true_n_startZ"])
true_mom = evt_df.AsNumpy(["px_end","py_end","pz_end"])
pred_mom = evt_df.AsNumpy(["px_end2","py_end2","pz_end2"])
# segment = evt_df


traj = {k: list(traj_point[k][0]) for k in traj_point}
ecal = {k: list(ecal_point[k][0]) for k in ecal_point}
seg = {k: list(seg_point[k][0]) for k in seg_point}

# true_n_startX"

# --- Griglia cilindri ---
theta = np.linspace(0, 2*np.pi, 100)
x = np.linspace(x_min, x_max, 50)
Theta, X = np.meshgrid(theta, x)

# --- Coordinate cilindri ---
Y_out = y_c + r_out * np.cos(Theta)
Z_out = z_c + r_out * np.sin(Theta)

Y_in  = y_c + r_in  * np.cos(Theta)
Z_in  = z_c + r_in  * np.sin(Theta)

# --- Creazione figura Plotly ---
fig = go.Figure()

# Cilindro esterno
fig.add_surface(
    x=X + x_c, y=Y_out, z=Z_out,
    colorscale='Blues', opacity=0.5, showscale=False,
    name='Outer cylinder'
)

# Cilindro interno
fig.add_surface(
    x=X + x_c, y=Y_in, z=Z_in,
    colorscale='Blues', opacity=0.5, showscale=False,
    name='Inner cylinder'
)

#vertex
fig.add_trace(go.Scatter3d(
    x= vtx_pos["vertex_x"], y=vtx_pos["vertex_y"], z=vtx_pos["vertex_z"],
    mode='markers+text',
    #text=labels,
    textposition='top center',
    marker=dict(size=6, color='orange', symbol='circle'),
    name='Vtx. position'
))

#firt hit
fig.add_trace(go.Scatter3d(
    x= p1["x1"], y=p1["y1"], z=p1["z1"],
    mode='markers+text',
    #text=labels,
    textposition='top center',
    marker=dict(size=6, color='green', symbol='circle'),
    name='p1 position'
))

#second hit
fig.add_trace(go.Scatter3d(
    x= p2["x2"], y=p2["y2"], z=p2["z2"],
    mode='markers+text',
    #text=labels,
    textposition='top center',
    marker=dict(size=6, color='green', symbol='circle'),
    name='p2 position'
))

#trajectory points
fig.add_trace(go.Scatter3d(
    x= traj["true_x_pos"], y=traj["true_y_pos"], z=traj["true_z_pos"],
    mode='markers',
    #text=labels,
    #textposition='top center',
    marker=dict(size=6, color='orange', symbol='circle'),
    name='Trajectory points'
))

#cluster position
fig.add_trace(go.Scatter3d(
    x= ecal["x_ecal"], y=ecal["y_ecal"], z=ecal["z_ecal"],
    mode='markers',
    #text=labels,
    #textposition='top center',
    marker=dict(size=6, color='blue', symbol='circle'),
    name='Ecal points'
))

#seg position
fig.add_trace(go.Scatter3d(
    x= seg["true_n_startX"], y=seg["true_n_startY"], z=seg["true_n_startZ"],
    mode='markers',
    #text=labels,
    #textposition='top center',
    marker=dict(size=6, color='pink', symbol='circle'),
    name='segment points'
))

x0 = vtx_pos["vertex_x"][0]
y0 = vtx_pos["vertex_y"][0]
z0 = vtx_pos["vertex_z"][0]

x1 = true_mom["px_end"][0]
y1 = true_mom["py_end"][0]
z1 = true_mom["pz_end"][0]
print(f"({x1}, {y1}, {z1})")
f = 5000
# x_new = x0 + f * (x1 - x0)
# y_new = y0 + f * (y1 - y0)
# z_new = z0 + f * (z1 - z0)

x_new = x0 + f * (x1)
y_new = y0 + f * (y1)
z_new = z0 + f * (z1)

fig.add_trace(go.Scatter3d(
    #x= true_mom["px_n"], y=true_mom["py_n"], z=true_mom["pz_n"],
    # x= vtx_pos["vertex_x"] + , y=vtx_pos["vertex_y"] + , z=vtx_pos["vertex_z"] + 
    # x=x_line + , y=y_line, z=z_line,
    # x= [vtx_pos["vertex_x"], true_mom["px_end"]] ,
    # y= [vtx_pos["vertex_y"], true_mom["py_end"]] ,
    # z= [vtx_pos["vertex_z"], true_mom["pz_end"]] ,

    # x=[x0, x1],
    # y=[y0, y1],
    # z=[z0, z1],

    x=[x0, x_new],
    y=[y0, y_new],
    z=[z0, z_new],

    mode='lines',
    #text=labels,
    #textposition='top center',
    line=dict(color='black', width=8),
    name='true neutron momentum'
))


x1_p = pred_mom["px_end2"][0]
y1_p = pred_mom["py_end2"][0]
z1_p = pred_mom["pz_end2"][0]

x_new2 = x0 + f * (x1_p)
y_new2 = y0 + f * (y1_p)
z_new2 = z0 + f * (z1_p)

#pred momentum
fig.add_trace(go.Scatter3d(
    x=[x0, x_new2],
    y=[y0, y_new2],
    z=[z0, z_new2],

    mode='lines',
    #text=labels,
    #textposition='top center',
    line=dict(color='red', width=8),
    name='pred neutron momentum'
))


# true_mom = evt_df.AsNumpy(["px_n","py_n","pz_n"])
# pred_mom = evt_df.AsNumpy(["n_px_pred", "n_py_pred","n_pz_pred"])


fig.update_layout(
    title='ECAL Cylinders with 3 Variable 3D Points',
    scene=dict(
        xaxis_title='X [mm]',
        yaxis_title='Y [mm]',
        zaxis_title='Z [mm]',
        aspectmode='data'
    ),
    width=900,
    height=700
)

fig.show()

(-0.16046595579481007, -0.8117232761826464, 0.5615656684076844)


In [ ]:
# --- Parametri cilindri (in mm) ---
r_out, r_in = 2230.0, 2000.0
x_min, x_max = -2150.0, 2150.0

# Centro del cilindro
x_c, y_c, z_c = 0.0, -2384.73, 23910.0

#event_index = 10
event_index = 153


evt_df2 = df_flag.Range(event_index,event_index+1)
# evt_df = (evt_df.Define("x1", "hit_pred[0]")
#           .Define("y1", "hit_pred[1]")
#           .Define("z1", "hit_pred[2]")
#           .Define("x2", "hit_pred[3]")
#           .Define("y2", "hit_pred[4]")
#           .Define("z2", "hit_pred[5]"))

evt_df2 = (evt_df2.Define("x1", "hit_pred2[0]")
          .Define("y1", "hit_pred2[1]")
          .Define("z1", "hit_pred2[2]")
          .Define("x2", "hit_pred2[3]")
          .Define("y2", "hit_pred2[4]")
          .Define("z2", "hit_pred2[5]")
          .Define("px_n", "n_px[0]")
          .Define("py_n", "n_py[0]")
          .Define("pz_n", "n_pz[0]")
          .Define("true_mom", "sqrt(px_n*px_n + py_n*py_n + pz_n*pz_n)" )
          .Define("px_end", "px_n/true_mom")
          .Define("py_end", "py_n/true_mom")
          .Define("pz_end", "pz_n/true_mom")
          .Define("pred_mom", "sqrt(n_px_pred*n_px_pred + n_py_pred*n_py_pred + n_pz_pred*n_pz_pred)" )
          .Define("px_end2", "n_px_pred/pred_mom")
          .Define("py_end2", "n_py_pred/pred_mom")
          .Define("pz_end2", "n_pz_pred/pred_mom")
          .Define("nu_true_mom", "sqrt(pxnu*pxnu + pynu*pynu + pznu*pznu)")
          .Define("px_nu", "pxnu/nu_true_mom")
          .Define("py_nu", "pynu/nu_true_mom")
          .Define("pz_nu", "pznu/nu_true_mom")
          .Define("mu_true_mom", "sqrt(mu_px[0]*mu_px[0] + mu_py[0]*mu_py[0] + mu_pz[0]*mu_pz[0])")
          .Define("px_mu", "mu_px[0]/mu_true_mom")
          .Define("py_mu", "mu_py[0]/mu_true_mom")
          .Define("pz_mu", "mu_pz[0]/mu_true_mom")
          
          )

vtx_pos = evt_df2.AsNumpy(["vertex_x","vertex_y","vertex_z"])
p1 = evt_df2.AsNumpy(["x1","y1","z1"])
p2 = evt_df2.AsNumpy(["x2","y2","z2"])
traj_point = evt_df2.AsNumpy(["true_x_pos", "true_y_pos", "true_z_pos"])
ecal_point = evt_df2.AsNumpy(["x_ecal", "y_ecal", "z_ecal"])
seg_point = evt_df2.AsNumpy(["true_n_startX", "true_n_startY", "true_n_startZ"])
true_mom = evt_df2.AsNumpy(["px_end","py_end","pz_end"])
pred_mom = evt_df2.AsNumpy(["px_end2","py_end2","pz_end2"])
true_mu_mom = evt_df2.AsNumpy(["px_mu","py_mu","pz_mu"])
true_nu_mom = evt_df2.AsNumpy(["px_nu","py_nu","pz_nu"])
# segment = evt_df


traj = {k: list(traj_point[k][0]) for k in traj_point}
ecal = {k: list(ecal_point[k][0]) for k in ecal_point}
seg = {k: list(seg_point[k][0]) for k in seg_point}

# true_n_startX"

# --- Griglia cilindri ---
theta = np.linspace(0, 2*np.pi, 100)
x = np.linspace(x_min, x_max, 50)
Theta, X = np.meshgrid(theta, x)

# --- Coordinate cilindri ---
Y_out = y_c + r_out * np.cos(Theta)
Z_out = z_c + r_out * np.sin(Theta)

Y_in  = y_c + r_in  * np.cos(Theta)
Z_in  = z_c + r_in  * np.sin(Theta)

# --- Creazione figura Plotly ---
fig = go.Figure()

# Cilindro esterno
fig.add_surface(
    x=X + x_c, y=Y_out, z=Z_out,
    colorscale='Blues', opacity=0.5, showscale=False,
    name='Outer cylinder'
)

# Cilindro interno
fig.add_surface(
    x=X + x_c, y=Y_in, z=Z_in,
    colorscale='Blues', opacity=0.5, showscale=False,
    name='Inner cylinder'
)

#vertex
fig.add_trace(go.Scatter3d(
    x= vtx_pos["vertex_x"], y=vtx_pos["vertex_y"], z=vtx_pos["vertex_z"],
    mode='markers+text',
    #text=labels,
    textposition='top center',
    marker=dict(size=6, color='black', symbol='circle'),
    name='Vtx. position'
))

#firt hit
# fig.add_trace(go.Scatter3d(
#     x= p1["x1"], y=p1["y1"], z=p1["z1"],
#     mode='markers+text',
#     #text=labels,
#     textposition='top center',
#     marker=dict(size=6, color='green', symbol='circle'),
#     name='p1 position'
# ))

#second hit
# fig.add_trace(go.Scatter3d(
#     x= p2["x2"], y=p2["y2"], z=p2["z2"],
#     mode='markers+text',
#     #text=labels,
#     textposition='top center',
#     marker=dict(size=6, color='green', symbol='circle'),
#     name='p2 position'
# ))

#trajectory points
# fig.add_trace(go.Scatter3d(
#     x= traj["true_x_pos"], y=traj["true_y_pos"], z=traj["true_z_pos"],
#     mode='markers',
#     #text=labels,
#     #textposition='top center',
#     marker=dict(size=6, color='orange', symbol='circle'),
#     name='Trajectory points'
# ))

# #cluster position
# fig.add_trace(go.Scatter3d(
#     x= ecal["x_ecal"], y=ecal["y_ecal"], z=ecal["z_ecal"],
#     mode='markers',
#     #text=labels,
#     #textposition='top center',
#     marker=dict(size=6, color='blue', symbol='circle'),
#     name='Ecal points'
# ))

# #seg position
# fig.add_trace(go.Scatter3d(
#     x= seg["true_n_startX"], y=seg["true_n_startY"], z=seg["true_n_startZ"],
#     mode='markers',
#     #text=labels,
#     #textposition='top center',
#     marker=dict(size=6, color='pink', symbol='circle'),
#     name='segment points'
# ))

x0 = vtx_pos["vertex_x"][0]
y0 = vtx_pos["vertex_y"][0]
z0 = vtx_pos["vertex_z"][0]

x1 = true_mom["px_end"][0]
y1 = true_mom["py_end"][0]
z1 = true_mom["pz_end"][0]
print(f"({x1}, {y1}, {z1})")
f = 5000
# x_new = x0 + f * (x1 - x0)
# y_new = y0 + f * (y1 - y0)
# z_new = z0 + f * (z1 - z0)

x_new = x0 + f * (x1)
y_new = y0 + f * (y1)
z_new = z0 + f * (z1)

fig.add_trace(go.Scatter3d(
    x=[x0, x_new],
    y=[y0, y_new],
    z=[z0, z_new],

    mode='lines',
    #text=labels,
    #textposition='top center',
    line=dict(color='green', width=8),
    name='true neutron momentum'
))

# x1_p = pred_mom["px_end2"][0]
# y1_p = pred_mom["py_end2"][0]
# z1_p = pred_mom["pz_end2"][0]

# x_new2 = x0 + f * (x1_p)
# y_new2 = y0 + f * (y1_p)
# z_new2 = z0 + f * (z1_p)

# #pred momentum
# fig.add_trace(go.Scatter3d(
#     x=[x0, x_new2],
#     y=[y0, y_new2],
#     z=[z0, z_new2],

#     mode='lines',
#     #text=labels,
#     #textposition='top center',
#     line=dict(color='red', width=8),
#     name='pred neutron momentum'
# ))



x1_nu = true_nu_mom["px_nu"][0]
y1_nu = true_nu_mom["py_nu"][0]
z1_nu = true_nu_mom["pz_nu"][0]

x_nu2 = x0 + f * (x1_nu)
y_nu2 = y0 + f * (y1_nu)
z_nu2 = z0 + f * (z1_nu)

#true nu momentum
fig.add_trace(go.Scatter3d(
    x=[x0, x_nu2],
    y=[y0, y_nu2],
    z=[z0, z_nu2],

    mode='lines',
    #text=labels,
    #textposition='top center',
    line=dict(color='black', width=8),
    name='true neutrino momentum'
))

x1_mu = float(true_mu_mom["px_mu"][0])
y1_mu = float(true_mu_mom["py_mu"][0])
z1_mu = float(true_mu_mom["pz_mu"][0])

# x1_mu = true_mu_mom["px_mu"][0]
# y1_mu = true_mu_mom["py_mu"][0]
# z1_mu = true_mu_mom["pz_mu"][0]

x_mu2 = x0 + f * (x1_mu)
y_mu2 = y0 + f * (y1_mu)
z_mu2 = z0 + f * (z1_mu)

#true nu momentum
fig.add_trace(go.Scatter3d(
    x=[x0, x_mu2],
    y=[y0, y_mu2],
    z=[z0, z_mu2],

    mode='lines',
    #text=labels,
    #textposition='top center',
    line=dict(color='blue', width=8),
    name='true muon momentum'
))




# fig.update_layout(
#     title='ECAL Cylinders with 3 Variable 3D Points',
#     scene=dict(
#         xaxis_title='X [mm]',
#         yaxis_title='Y [mm]',
#         zaxis_title='Z [mm]',
#         aspectmode='data'
#     ),
#     width=900,
#     height=700
# )

fig.show()

(0.5139621021437826, -0.40557612959443773, -0.755877609579183)


TypeError: float() argument must be a string or a number, not 'RVec<double>'